# 初級①：データ探索（EDA）＋可視化

このセクションでは、GOLD テーブルを使って回答データを探索し、  
集計・可視化を通じて傾向を把握します。

目的：
- 属性（年代・性別・地域など）別満足度の傾向を見る  
- ネガティブコメントカテゴリの分布を確認  
- 地域ごとのネガティブカテゴリ構成を比較  

In [0]:
CATALOG = "hiroshi"
SCHEMA  = "survey_analysis_simple"

# UCオブジェクト作成
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {SCHEMA}")

GOLD_TABLE_NAME = "gold_survey_responses_final"

## GOLD テーブル読み込み・構造確認

このセルでは、GOLD テーブルを DataFrame として読み込み、  
冒頭の数行とスキーマを表示して、どの列があるか、型は何かを確認します。

In [0]:
gold = spark.table(GOLD_TABLE_NAME)  # ← 実際の GOLD テーブル名
display(gold.limit(10))
gold.printSchema()

In [0]:
from pyspark.sql import functions as F

agg_df = (
    gold
    .groupBy("age_group", "gender")
    .agg(
        F.count("*").alias("count"),
        F.avg("satisfaction_int").alias("avg_satisfaction")
    )
    .orderBy("age_group", "gender")
)

display(agg_df)

## 年代 × 性別 ごとの満足度平均を集計・可視化準備

このセルでは、`age_group` × `gender` ごとに件数と平均満足度 (`satisfaction_int`) を算出し、  
その結果を表示します（後続セルで可視化できます）。

In [0]:
from pyspark.sql import functions as F

agg_df = (
    gold
    .groupBy("age_group", "gender")
    .agg(
        F.count("*").alias("count"),
        F.avg("satisfaction_int").alias("avg_satisfaction")
    )
    .orderBy("age_group", "gender")
)

display(agg_df)

## 属性別満足度の棒グラフ可視化

このセルでは、先ほど集計した `agg_df` を使って、  
ノートブック UI の `display()` により棒グラフ・チャート表示を可能にします。  
チャートは “Bar chart” を選んで属性別満足度傾向を可視化します。

In [0]:
# display(agg_df) で DataFrame 表示済みなので、UI 上で Bar chart を選択すればグラフ表示される
# 追加：チャート表示を再度強制するため、display(agg_df) をもう一度
display(agg_df)

## ネガティブコメントカテゴリの頻度集計・可視化準備

このセルでは、`negative_feedback_category` ごとの出現件数を集計し、  
カテゴリ分布を把握できる DataFrame を生成して表示します。

In [0]:
neg_df = (
    gold
    .groupBy("negative_feedback_category")
    .agg(F.count("*").alias("cnt"))
    .orderBy(F.desc("cnt"))
)

display(neg_df)

## ネガティブカテゴリ出現頻度の可視化

このセルでは `neg_df` を使って、棒グラフなどでカテゴリの頻度分布を可視化します。  
ノートブック UI で “Visualization → Bar chart” を選択することで視覚化できます。

In [0]:
display(neg_df)

## 地域別 × ネガティブカテゴリ割合の比較集計

このセルでは、地域ごとにネガティブカテゴリごとの件数を出し、  
その地域の総件数で割って割合（ratio）を出します。  
これにより、地域間で“不満点の種類”に違いがあるかを見やすくします。

In [0]:
# 地域別カテゴリ別集計
region_cat = (
    gold
    .groupBy("region", "negative_feedback_category")
    .agg(F.count("*").alias("cnt"))
)

# 地域別総件数
region_tot = (
    gold
    .groupBy("region")
    .agg(F.count("*").alias("total_cnt"))
)

# 結合して割合列を計算
region_cat_ratio = (
    region_cat
    .join(region_tot, on="region")
    .withColumn("ratio", F.col("cnt") / F.col("total_cnt"))
    .orderBy("region", F.desc("ratio"))
)

display(region_cat_ratio)

## ✅ まとめ・次のステップ

この章で得られる主な知見例：
- 属性別（年代・性別など）の満足度傾向  
- ネガティブコメントカテゴリの頻出度  
- 地域ごとの不満カテゴリ構成の差異  

次の章（初級② など）では、この集計基盤を使って以下に進めます：
- 属性とネガティブカテゴリのクロス集計  
- SparkML による「購入意向予測モデル」の構築  
- LLM を使った要望文句の意味解析  

この章で可視化・集計を通じて得た理解が、次の分析フェーズの基盤になります。